In [105]:
import re
import subprocess
import os
import json

import pandas as pd

# import xlwings as xw


In [106]:
exec_name = "cyber"
dir = "out/" + exec_name + "/"
dir_md = dir + "comands-md-files/"


In [107]:
if not os.path.isdir(dir):
    os.mkdir(dir)
    os.mkdir(dir_md)

for f in os.listdir(dir_md):
    os.remove(os.path.join(dir_md, f))


In [108]:
chapters_to_include = [
    "Available Commands:",
    "Usage:",
    "Aliases:",
    "Flags:",
    "Global Flags:",
    "Example:",
    "Examples:",
]


In [109]:
def get_cli_output(command: str):

    _args = command.split()
    _args.append("-h")

    out = subprocess.run(_args, capture_output=True)
    text = out.stdout.decode("utf-8") + out.stderr.decode("utf-8")

    with open(dir_md + command + ".md", "w") as file:
        file.write(text)

    return text


# text = get_cli_output('cyber')


In [110]:
def parse_cli_stdout(command: str):

    _text = get_cli_output(command)
    _lines = _text.split("\n")

    chapters = {"command": command}
    chapter = "Description"
    chapters[chapter] = []

    for l in _lines:
        if l != "":
            if l in chapters_to_include:
                chapter = l[:-1]  # cutting ":"
                chapters[chapter] = []
            else:
                chapters[chapter].append(l)

    return chapters


# chapters = parse_cli_stdout("gaiad")
# print(chapters)


In [111]:
def get_sub_commands(chapters):
    new_commands = []

    if "Available Commands" in chapters:
        for l in chapters["Available Commands"]:
            if _match := re.search(r"^\s+(.*?)\s+(.*)", l):
                sub_command = _match.group(1)

                command_candidate = " ".join(
                    (chapters["command"] + " " + sub_command).split()
                )
                new_commands.append(command_candidate)

    return new_commands


# new_commands = get_sub_commands(chapters)
# new_commands


In [112]:
parsed_commands = []
all_commands = []
new_commands = [exec_name + " "]

while True:
    for command in new_commands:

        commands_dict = parse_cli_stdout(command)
        new_command_candidates = get_sub_commands(commands_dict)
        all_commands.append(commands_dict)

        for candidate in new_command_candidates:
            if candidate not in parsed_commands:
                new_commands.append(candidate)
                parsed_commands.append(candidate)
        new_commands.remove(command)
    if new_commands == []:
        break


In [113]:
# all_commands


In [114]:
json_object = json.dumps(all_commands, indent=4)

with open(dir + exec_name + "_commands.json", "w") as outfile:
    outfile.write(json_object)


In [115]:
len(all_commands)


284

In [116]:
command_files = []

for f in os.listdir(dir_md):
    if f[-3:] == ".md":
        command_files.append(f)

command_files.sort()
# command_files


In [117]:
if os.path.exists(dir + exec_name + "_commands.md"):
    os.remove(dir + exec_name + "_commands.md")

for f in command_files:
    with open(dir_md + f) as f2:
        text = f2.read()
        file_content = f"### {f[:-3]}\n\n```\n{text}\n```\n\n"

        with open(dir + exec_name + "_commands.md", "a") as file:
            file.write(file_content)


In [118]:
# with open("all_commands.json", "r") as file:
#     all_commands = json.load(file)


In [119]:
commands = []

for c in all_commands:
    if "Available Commands" not in c:
        commands.append(c)


In [120]:
count = 0
for c in commands:
    all_flags = []
    all_flags.extend(c["Flags"])
    del c["Flags"]
    if "Global Flags" in c:
        all_flags.extend(c["Global Flags"])
        del c["Global Flags"]

    if all_flags != []:
        c["flags_parsed"] = []
        for b in all_flags:
            if match1 := re.match(
                r"\s\s+(?:-(?P<short>\w)[,\s]+)?(?:--(?P<long>[.\w-]+))\s(?:(?P<format>(uint32|ints|strings|int 64Slice|int|string|uint|float|bytesHex)))?\s*(?P<description>.*)?",
                b,
            ):
                c["flags_parsed"].append(
                    {
                        "long": match1.group("long"),
                        "short": match1.group("short"),
                        "format": match1.group("format"),
                        "description": match1.group("description"),
                    }
                )


In [121]:
for c in commands:
    if len(c["Usage"]) > 1:
        print(c["command"], "command has more than 1 usages", c["Usage"])
    else:
        c["Usage"] = c["Usage"][0]


In [122]:
for entry in commands[:5]:
    print(entry["Usage"])


  cyber collect-gentxs [flags]
  cyber gentx [key_name] [amount] [flags]
  cyber init [moniker] [flags]
  cyber migrate [target-version] [genesis-file] [flags]
  cyber rollback [flags]


In [123]:
usage = "'--type=[hash|acc_seq|signature] [hash|acc_seq|signature]'"
re.sub(r"--.*?( |=)(<|\[).*?(>|\])", "", usage)[0]


"'"

In [124]:
# geting rid of --flags before neccessary arg
# cyber query tx ['--type=[hash|acc_seq|signature] [hash|acc_seq|signature]']

for c in commands:

    usage = c["Usage"]
    usage = usage.replace("[flags]", "").replace(c["command"], "")
    usage = re.sub(r"--.*?( |=)(<|\[).*?(>|\])", "", usage) 
    usage = re.sub(r"\s+", " ", usage).strip()

    c["Usage"] = usage


In [125]:
# for c in commands:
#     print(c["command"], c["Usage"])


cyber collect-gentxs 
cyber gentx [key_name] [amount]
cyber init [moniker]
cyber migrate [target-version] [genesis-file]
cyber rollback 
cyber status 
cyber testnet 
cyber validate-genesis [file]
cyber debug addr [address]
cyber debug raw-bytes [raw-bytes]
cyber add-genesis-account [address_or_key_name] [coin][,[coin]]
cyber export 
cyber start 
cyber debug pubkey [pubkey]
cyber keys delete <name>...
cyber keys import <name> <keyfile>
cyber keys migrate <old_home_dir>
cyber keys parse <hex-or-bech32-address>
cyber tx broadcast [file_path]
cyber tx decode [amino-byte-string]
cyber tx encode [file]
cyber tx multisign-batch [file] [name] [[signature-file]...]
cyber tx sign [file]
cyber tx validate-signatures [file]
cyber tx authz grant <grantee> <authorization_type="send"|"generic"|"delegate"|"unbond"|"redelegate">
cyber tx feegrant grant [granter_key_or_address] [grantee]
cyber tx graph cyberlink [cid-from] [cid-to]
cyber tx liquidity deposit [pool-id] [deposit-coins]
cyber tx liquidity 

In [126]:
commands.sort(key=lambda x: x["command"])


In [139]:
# removing optional parameters in brackets ['[name] (--upgrade-height [height]) (--upgrade-info [info])']

arguments = {}

for c in commands:
    # clean_string = c["Usage"].split("(")[0].split("--")[0]
    clean_string = c["Usage"].split("(")[0]
    print(c["command"], clean_string)
    arguments[c["command"]] = clean_string.split(" ")

    args = re.findall(r"(?:\[|\<)([\w|]+?)(?:\]|\>)(\?)?", clean_string)
    c["args"] = []
    for p in args:
        p = "".join(p)
        p = re.sub(r"[^A-Za-z0-9]", "_", p)
        c["args"].append(p)


cyber add-genesis-account [address_or_key_name] [coin][,[coin]]
cyber collect-gentxs 
cyber config <key> [value]
cyber debug addr [address]
cyber debug pubkey [pubkey]
cyber debug raw-bytes [raw-bytes]
cyber export 
cyber gentx [key_name] [amount]
cyber help [command]
cyber init [moniker]
cyber keys add <name>
cyber keys delete <name>...
cyber keys export <name>
cyber keys import <name> <keyfile>
cyber keys list 
cyber keys migrate <old_home_dir>
cyber keys mnemonic 
cyber keys parse <hex-or-bech32-address>
cyber keys show [name_or_address [name_or_address...]]
cyber migrate [target-version] [genesis-file]
cyber query account [address]
cyber query auth account [address]
cyber query auth accounts 
cyber query auth params 
cyber query authz grants [granter-addr] [grantee-addr] [msg-type-url]?
cyber query authz grants-by-grantee [grantee-addr]
cyber query authz grants-by-granter [granter-addr]
cyber query bandwidth load 
cyber query bandwidth neuron [address]
cyber query bandwidth params 

In [141]:
commands

[{'command': 'cyber add-genesis-account',
  'Description': ['Add a genesis account to genesis.json. The provided account must specify',
   'the account address or key name and a list of initial coins. If a key name is given,',
   'the address will be looked up in the local Keybase. The list of initial tokens must',
   'contain valid denominations. Accounts may optionally be supplied with vesting parameters.'],
  'Usage': '[address_or_key_name] [coin][,[coin]]',
  'flags_parsed': [{'long': 'height',
    'short': None,
    'format': 'int',
    'description': 'Use a specific height to query state at (this can error if the node is pruning state)'},
   {'long': 'help',
    'short': 'h',
    'format': None,
    'description': 'help for add-genesis-account'},
   {'long': 'keyring-backend',
    'short': None,
    'format': 'string',
    'description': 'Select keyring\'s backend (os|file|kwallet|pass|test) (default "os")',
    'completions_fun': '"nu-completions-cyber--os-file-kwallet-pass-test

In [142]:
def make_fun_name(text: str):
    rsub = re.sub(r"([^a-z])", "-", text)
    return f'"nu-completions-{exec_name}-{rsub}"'


completions_list = []

for c in commands:
    for f in c["flags_parsed"]:
        if m := re.search(r"(\(.*\|.*?\))", f["description"]):
            completions_list.append(m[0])
            f["completions_fun"] = make_fun_name(m[0])


commands[:10]


[{'command': 'cyber add-genesis-account',
  'Description': ['Add a genesis account to genesis.json. The provided account must specify',
   'the account address or key name and a list of initial coins. If a key name is given,',
   'the address will be looked up in the local Keybase. The list of initial tokens must',
   'contain valid denominations. Accounts may optionally be supplied with vesting parameters.'],
  'Usage': '[address_or_key_name] [coin][,[coin]]',
  'flags_parsed': [{'long': 'height',
    'short': None,
    'format': 'int',
    'description': 'Use a specific height to query state at (this can error if the node is pruning state)'},
   {'long': 'help',
    'short': 'h',
    'format': None,
    'description': 'help for add-genesis-account'},
   {'long': 'keyring-backend',
    'short': None,
    'format': 'string',
    'description': 'Select keyring\'s backend (os|file|kwallet|pass|test) (default "os")',
    'completions_fun': '"nu-completions-cyber--os-file-kwallet-pass-test

In [143]:
nu_functions_list = []

for i in set(completions_list):
    fun_list = (
        i.replace(" ", "")
        .replace("(", '{ ["')
        .replace("|", '", "')
        .replace(")", '"] }')
    )
    fun_name = make_fun_name(i)
    fun_complete = f"def {fun_name} [] {fun_list}"
    nu_functions_list.append(fun_complete)

nu_completions_functions_string = "\n".join(nu_functions_list)


In [144]:
re.findall(
    r"(\(.*\|.*?\))",
    'Select keyring\'s backend (os|file|kwallet|pass|test) (default "os")',
)


['(os|file|kwallet|pass|test)']

In [145]:
# df = pd.DataFrame(commands)
# pd.json_normalize(df['flags_parsed'])


In [146]:
# df = pd.DataFrame(commands).explode('flags_parsed').reset_index()
# df['command'] = df['command'].apply(lambda x: str(x))
# df2 = pd.json_normalize(df['flags_parsed'])

# df3 = pd.concat([df[['command', 'Usage']], df2], axis=1)
# xw.view(df3)


In [147]:
# for_excel = pd.DataFrame(data=commands).applymap(lambda x: str(x))
# xw.view(for_excel)


In [148]:
# for i in all_commands:
#     print(i["command"], "\n", i["flags_parsed"])


In [149]:
types_nu_dict = {
    "uint": "int",
    "uint32": "int",
    # "float": "decimal",
    "float": "string",
    "strings": "string",
    "bytesHex": "string",
    "ints": "string",
}

# nu functions to add at the beginning of the file
functions_list = f"""def "{exec_name} _keys table" [] {{
	{exec_name} keys list | lines | window 5 -s 5 | 
    each {{|it| ($it| parse -r '(?P<col>\w+):(?P<value>.*)')}} | 
    each {{|it| ($it| transpose -r)}} | reduce {{|it, acc| $it | append $acc}} | 
    select name type address 
}}

def "nu-complete {exec_name} _keys names" [] {{
    ({exec_name} keys table).name | zip ({exec_name} _keys table).address | flatten
  }}

"""

lines = [functions_list, nu_completions_functions_string]

for c in commands:
    description = " ".join(c["Description"])[:400]

    lines.append(f"\n# {description}\nextern '{c['command']}' [\n")

    for arg in c["args"]:
        lines.append(f"\t{arg}: string\n")

    for flags in c["flags_parsed"]:
        if "." not in flags["long"]:
            long = flags["long"]
            short = "(-" + flags["short"] + ")" if flags["short"] else ""
            f1 = types_nu_dict.get(flags["format"], flags["format"])
            if "completions_fun" in flags:
                f1 = f1 + "@" + flags["completions_fun"]
            format = ": " + f1 if f1 else ""
            description = flags["description"]
            lines.append(f"\t--{long}{short}{format}\t\t# {description}\n")
    lines.append("]")

lines = [
    l.replace("--from: string", f'--from: string@"nu-complete {exec_name} _keys names"')
    for l in lines
]


with open(dir + exec_name + "_nu_completions.nu", "w") as file:
    file.writelines(lines)
